In [4]:
from kiteconnect import KiteConnect
import hashlib
import pandas as pd
import re

In [5]:

# Step 1: Initialize KiteConnect
api_key = "6wgmb42mve0cz6aa"
api_secret = "smwerv7dpgxpeyr71f2eu65hzqjlqf3l"

kite = KiteConnect(api_key=api_key)

# Step 2: Get the login URL
print("Login URL:", kite.login_url())

Login URL: https://kite.zerodha.com/connect/login?api_key=6wgmb42mve0cz6aa&v=3


In [6]:
request_token = "4ssf14R3R0xKsYYchbdAB9vTUOAe64Jr"

try:

    session_data = kite.generate_session(request_token=request_token, api_secret=api_secret)
    access_token = session_data["access_token"]
    
    # Print the access token
    print("Access Token:", access_token)
except Exception as e:
    print("Error generating access token:", str(e))

Access Token: fOr5cezL3YjH3siCM0Q4iOSrYUdLaDEP


In [5]:
kite.set_access_token(access_token=access_token)


In [7]:
instruments = kite.instruments()  # returns a list of dicts
df_instruments = pd.DataFrame(instruments)


In [8]:
df = pd.DataFrame(instruments)
df.head()


,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,223969541,874881,BANKEX25NOVFUT,BANKEX,0.0,2025-11-27,0.0,0.05,30,FUT,BFO-FUT,BFO
1,282664965,1104160,BANKEX25DECFUT,BANKEX,0.0,2025-12-24,0.0,0.05,30,FUT,BFO-FUT,BFO
2,293244165,1145485,BANKEX26JANFUT,BANKEX,0.0,2026-01-29,0.0,0.05,30,FUT,BFO-FUT,BFO
3,223700485,873830,SENSEX25NOVFUT,SENSEX,0.0,2025-11-27,0.0,0.05,20,FUT,BFO-FUT,BFO
4,282186245,1102290,SENSEX25DECFUT,SENSEX,0.0,2025-12-24,0.0,0.05,20,FUT,BFO-FUT,BFO


In [9]:
equity_df = df[df['segment'].isin(['NSE', 'BSE'])]
equity_df.head()


,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
4855,50948356,199017,IVZGOLDINAV,INAV INVESCINDGOLXCHANGER,0.0,,0.0,0.01,1,EQ,BSE,BSE
4856,50948612,199018,HDFCETFINAV,INAVHDFCMUTUAL-HDFC GOLD E,0.0,,0.0,0.01,1,EQ,BSE,BSE
4857,50948868,199019,ICICIGDINAV,INAVICICI PRUDENTIAL GOLD ETF,0.0,,0.0,0.01,1,EQ,BSE,BSE
4858,50949124,199020,MON100INAV,INAVMOTILALOSWALMOSTSHARNASD,0.0,,0.0,0.01,1,EQ,BSE,BSE
4859,50949380,199021,BSLGOLDINAV,INAVADITYABIRLASUNLIFMUTUAL,0.0,,0.0,0.01,1,EQ,BSE,BSE


In [10]:
total_equity = len(equity_df)
total_equity


21214

In [11]:
equity_df['segment'].value_counts()


segment
BSE    12753
NSE     8461
Name: count, dtype: int64

In [12]:
equity_df['instrument_type'].value_counts()


instrument_type
EQ    21214
Name: count, dtype: int64

In [13]:
unique_stocks = equity_df['tradingsymbol'].nunique()
unique_stocks


19089

In [14]:
# change path if your file is elsewhere
nifty_file = "./nifty500.txt"

with open(nifty_file, "r", encoding="utf-8") as f:
    nifty_lines = [line.strip() for line in f if line.strip()]

# Turn into a list and upper-case (safe-guard)
nifty_tickers_raw = [s.upper() for s in nifty_lines]
len(nifty_tickers_raw), nifty_tickers_raw[:10]   # quick sanity


(502,
 ['360ONE',
  '3MINDIA',
  'AADHARHFC',
  'AARTIIND',
  'AAVAS',
  'ABB',
  'ABBOTINDIA',
  'ABCAPITAL',
  'ABFRL',
  'ABLBL'])

In [15]:
def normalize_sym(s):
    # uppercase, remove spaces, dots, ampersands, slashes, parentheses, and hyphens
    s = str(s).upper()
    s = re.sub(r'[\s\.\&\/\(\)\,]', '', s)
    s = s.replace('-', '')   # keep hyphen removal explicit
    return s

# create normalized columns
df_instruments['norm_tradingsymbol'] = df_instruments['tradingsymbol'].astype(str).apply(normalize_sym)
nifty_norm = [normalize_sym(t) for t in nifty_tickers_raw]


In [17]:
# Filter only NSE traded rows (most nifty constituents are on NSE)
nse_df = df_instruments[df_instruments['exchange'].str.upper().eq('NSE')].copy()

# Keep only rows whose normalized symbol is in your normalized nifty list
nifty_df = nse_df[nse_df['norm_tradingsymbol'].isin(nifty_norm)].copy()

# Drop duplicates (some tickers may appear multiple times with different token types)
nifty_df = nifty_df.drop_duplicates(subset=['norm_tradingsymbol', 'exchange', 'instrument_token'])

nifty_df.head()


,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange,norm_tradingsymbol
109012,1793,7,AARTIIND,AARTI INDUSTRIES,0.0,,0.0,0.05,1,EQ,NSE,NSE,AARTIIND
109013,3329,13,ABB,ABB INDIA,0.0,,0.0,0.50,1,EQ,NSE,NSE,ABB
109020,5633,22,ACC,ACC,0.0,,0.0,0.10,1,EQ,NSE,NSE,ACC
109022,6401,25,ADANIENT,ADANI ENTERPRISES,0.0,,0.0,0.10,1,EQ,NSE,NSE,ADANIENT
109029,10241,40,AEGISLOG,AEGIS LOGISTICS,0.0,,0.0,0.05,1,EQ,NSE,NSE,AEGISLOG


In [18]:
print("Total nifty tickers in your file:", len(nifty_tickers_raw))
print("Matched rows found in instruments:", len(nifty_df))
print("Unique matched tickers:", nifty_df['norm_tradingsymbol'].nunique())

# show a clean preview
cols_show = ['tradingsymbol','name','exchange','segment','instrument_token','lot_size','expiry','tick_size']
for c in cols_show:
    if c not in nifty_df.columns:
        cols_show.remove(c)   # safe-guard if column missing

nifty_df[cols_show].sort_values('tradingsymbol').head(40)


Total nifty tickers in your file: 502
Matched rows found in instruments: 498
Unique matched tickers: 498


,tradingsymbol,name,exchange,segment,instrument_token,lot_size,expiry,tick_size
111772,360ONE,360 ONE WAM,NSE,NSE,3343617,1,,0.10
109157,3MINDIA,3M INDIA,NSE,NSE,121345,1,,5.00
114699,AADHARHFC,AADHAR HOUSING FINANCE L,NSE,NSE,6074625,1,,0.05
109012,AARTIIND,AARTI INDUSTRIES,NSE,NSE,1793,1,,0.05
110291,AAVAS,AAVAS FINANCIERS,NSE,NSE,1378561,1,,0.10
109013,ABB,ABB INDIA,NSE,NSE,3329,1,,0.50
112837,ABBOTINDIA,ABBOTT INDIA,NSE,NSE,4583169,1,,5.00
113802,ABCAPITAL,ADITYA BIRLA CAPITAL,NSE,NSE,5533185,1,,0.05
116153,ABFRL,ADITYA BIRLA FASHION & RT,NSE,NSE,7707649,1,,0.01
116478,ABLBL,ADITYA BIRLA LIFES BRAN L,NSE,NSE,193751809,1,,0.01


In [19]:
out_path = "./data/nifty500_instruments_matched.csv"
nifty_df.to_csv(out_path, index=False)
print("Saved matched instruments to:", out_path)


Saved matched instruments to: ./data/nifty500_instruments_matched.csv
